# Using BFGS to obtain MAP estimate

In [14]:
from scipy.sparse import lil_matrix
import edward as ed
from edward.models import Beta, Gamma, Exponential
import tensorflow as tf
import numpy as np
import math
import pandas as pd

In [16]:
sess = ed.get_session()

## Define Functions

In [18]:
def build_cascade(time, seed, T):
    # Store number of nodes
    n = time.shape[0]

    # Transpose times and reduce minimum
    times_T = tf.minimum(tf.transpose(time),T)

    # Initialize transmission times to be max time except for seed node
    transmission = tf.ones(n)*T
    transmission = tf.subtract(transmission,tf.one_hot(seed, n)*T)

    
    # Continually update transmissions
    for _ in range(n):

        # Tile transmission
        transmission_tiled = tf.reshape(tf.tile(transmission,[n]),[n,n])

        # Add transposed times and tiled transmissions
        potential_transmission = tf.add(transmission_tiled,times_T)

        # Find minimum path from all new 
        potential_transmission_row = tf.reduce_min(potential_transmission, reduction_indices=[1])

        # Concatenate previous transmission and potential new transmission
        potential_transmission_stack = tf.stack([transmission,potential_transmission_row],axis=0)

        # Take the minimum of the original transmission and the potential new transmission
        transmission = tf.reduce_min(potential_transmission_stack, reduction_indices=[0])

    return transmission

## Generate Data

In [22]:
alpha = np.array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=np.float32)

alpha_tf = tf.convert_to_tensor(alpha, dtype=tf.float32)

tau = Exponential(alpha)

sess.run(build_cascade(tau, 0, 10))


array([0.       , 2.147508 , 2.2670565, 3.3984096, 3.8358562, 3.8827097,
       7.4505334, 7.627615 , 7.9959984, 8.679662 ], dtype=float32)

In [ ]:
l = []
for i in tqdm(range(100)):
    for i in range(10):
        x = sess.run(build_cascade(tau, i, 10))
        l.append(x)
data = np.vstack(l)

## Load the data (from the previous step)

In [8]:
data = pd.read_csv('test-data.csv').values

## Define the BFGS model

In [9]:
def cascade_bfgs(d):
    c = d.shape[0]
    N = d.shape[1]
    max_iter = 1000

    ## Initialize the values 

    # Cascades should be of size c x n; Initialize the variables

    C = tf.placeholder(tf.float32, [c,N])
    B = tf.Variable(tf.random_normal([N,N]), dtype=tf.float32)
    A = tf.nn.softmax(B)

    #A = tf.Variable(tf.ones([N,N], dtype=tf.float32), dtype=tf.float32)

    I_inf = tf.expand_dims(tf.where(C>0, x=tf.ones((c,N)), y=tf.zeros((c,N))),1)
    I_uninf = 1 - tf.expand_dims(tf.where(C>0, x=tf.ones((c,N)), y=tf.zeros((c,N))),1)
    t_max = tf.expand_dims(tf.reduce_max(C,1),1)
    #calculate dist1
    t_mi = tf.expand_dims(tf.subtract(t_max,C),1)
    #calculate dist2
    k = tf.concat([-tf.ones((c,N,1)),tf.expand_dims(C,-1)], 2)
    k_T = tf.transpose(tf.concat([tf.expand_dims(C,-1),tf.ones((c,N,1))], 2),perm=[0,2,1])
    t_ij = tf.transpose(tf.matmul(k,k_T),perm=[0,2,1])
    # calculate psi_1 
    psi_1 = tf.multiply(tf.multiply(A,I_inf), tf.transpose(I_uninf, perm=[0,2,1]))
    psi_1 = tf.reduce_sum(-tf.multiply(psi_1,t_mi))
    # calculate psi_2
    psi_2 = tf.multiply(tf.multiply(A,I_inf), tf.transpose(I_inf, perm=[0,2,1]))
    psi_2 = -tf.multiply(psi_2,t_ij)
    psi_2 = tf.reduce_sum(tf.where(t_ij > 0, psi_2,tf.zeros((c,N,N))))

    # calculate psi_3
    psi_3 = tf.multiply(tf.multiply(A,I_inf), tf.transpose(I_inf, perm=[0,2,1]))
    psi_3 = tf.reduce_sum(tf.where(t_ij > 0, psi_3,tf.zeros((c,N,N))), axis=1)
    psi_3 = tf.reduce_sum(tf.where(tf.is_inf(psi_3), tf.zeros_like(psi_3), psi_3))


    # log(p)

    log_p = -(psi_1 + psi_2 + psi_3)

    #c_i = tf.exp(log_p)
    data = {C: d}

    optimizer = tf.contrib.opt.ScipyOptimizerInterface(log_p,
                                                       method='L-BFGS-B',
                                                       options={
                                                        'maxiter': max_iter})
    model = tf.global_variables_initializer()
    sess = tf.Session()

    sess.run(model)
    optimizer.minimize(sess, feed_dict=data)
    b = B.eval(session=sess)
    a = A.eval(session=sess)
    return a, b

## Run the model 

In [26]:
data[data == 10] = 0

In [28]:
a, b = cascade_bfgs(data)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: -57.321350
  Number of iterations: 9
  Number of functions evaluations: 24


## Evaluate

In [31]:
a.round()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [33]:
alpha - a.round()

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
       [-1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)